# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902027


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:01<00:26,  1.09it/s]

Rendering models:  13%|█▎        | 4/31 [00:07<00:38,  1.44s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:27,  1.07s/it]

Rendering models:  23%|██▎       | 7/31 [00:07<00:18,  1.27it/s]

...

Rendering models:  90%|█████████ | 28/31 [00:11<00:00,  4.07it/s]

Rendering models:  94%|█████████▎| 29/31 [00:11<00:00,  3.78it/s]

Rendering models:  97%|█████████▋| 30/31 [00:11<00:00,  4.09it/s]

Rendering models: 100%|██████████| 31/31 [00:11<00:00,  4.20it/s]

equidad1                              0.000939
BeNjEsSePh                            0.000935
nickoftona                            0.001033
kayleebug2017                         0.003096
23williamrm                           0.003140
Linda_J._Berkel                       0.000760
not-logged-in-0b0736c8991a7cc2017a    0.002171
Thedunkmasta                          0.002609
not-logged-in-a5721f5f24fded19f7ed    0.001075
tingard                               0.001040
Raven_Vasquez                         0.001132
crush202020                           0.001449
pangeli5                              0.000906
Planetme                              0.001381
peakscience7                          0.000604
not-logged-in-9f30f411df9d28d2c541    0.005755
ShirleyL                              0.000694
not-logged-in-a77c0eef6525bed230df    0.000777
Mr_DMan                               0.001416
not-logged-in-a7a1a71edf4e4a2f24bc    0.005259
ElisabethB                            0.004941
helios1235   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())